In [1]:
import pandas as pd
import re

In [2]:
course_2019_2020 = pd.read_csv('../data/course_2019.csv', header=0, encoding='unicode_escape')
course_2022_2023 = pd.read_csv('../data/course_2022.csv', header=0, encoding='unicode_escape')

In [11]:
def group_by_year(df):
    grouped_df = df.groupby(['Subj','Course', 'Title','Term', 'Term Yr', 'Class Yr', 'Dept Desc']).agg({'Cap': 'sum', 'Enrolled': 'sum'}).reset_index()

    return grouped_df

def drop_unneeded_cols(df):
    df['Course'] = df.apply(lambda row: row['Subj'] + ' ' + row['Num'], axis=1)

    df.drop(columns=['CRN', 'Num', 'Section', 'Instructor', 'Camp', 'Bldg', 'Room','Units', 'Begin', 'End', 'Days', 'Start Date', 'End Date', 'Faculty', 'Status'], inplace=True)

    return df

def drop_out_of_scope(df):
    df.dropna(subset=['Faculty'], inplace=True)
    df = df[df['Faculty'] == 'EN']

    df = df[~df['Section'].str.contains('B|T|W')]

    df.drop(['Sched Type'], axis=1, inplace=True)

    df = df[~df['Subj'].str.contains('BME|CIVE|MECH')]

    return df

def drop_non_stream(df):
    SENG_DEGREE = {
        "YEAR_1": ["CSC 111", "CSC 115"],
        "YEAR_2": ["ECE 255", "CSC 230", "ECE 260", "SENG 265", "CSC 225", "ECE 310", "SENG 275", "SENG 310"] ,
        "YEAR_3": ["ECE 458", "CSC 361", "CSC 226", "ECE 360", "SENG 321", "SENG 371", "ECE 355", "CSC 355", "CSC 320", "CSC 360", "CSC 370", "SENG 350", "SENG 360"],
        "YEAR_4": ["SENG 426", "SENG 440", "SENG 499", "ECE 455", "CSC 460", "SENG 401"]
    }

    # only allow those courses that are in the SENG degree
    df = df[df['Course'].isin(SENG_DEGREE['YEAR_1'] + SENG_DEGREE['YEAR_2'] + SENG_DEGREE['YEAR_3'] + SENG_DEGREE['YEAR_4'])]

    return df

def assign_yrs(df):
    df['Class Yr'] = df['Num'].apply(lambda id: int(re.search(r'\d', str(id)).group()) if int(re.search(r'\d', str(id)).group()) < 5 else None)
    df = df.dropna(subset=['Class Yr'])
    df['Class Yr'] = df['Class Yr'].astype(int)
    
    return df

def split_term_year(df):
    df['Term Yr'] = df['Term'].apply(lambda term: int(re.search(r'^\d{4}', str(term)).group()))

    return df

def split_term(df):
    term_mapping = {
        '01': 'Spring',
        '05': 'Summer',
        '09': 'Fall'
    }
    df['Term'] = df['Term'].apply(lambda term: term_mapping[re.search(r'\d{2}$', str(term)).group()])

    return df

def reassign_seng(df):
    # if the course start with SENG, then set the Dept Desc as Engineering & Computer Science
    df['Dept Desc'] = df.apply(lambda row: 'Engineering & Computer Science' if row['Subj'] == 'SENG' else row['Dept Desc'], axis=1)

    return df

In [12]:
year_2019_2020 = drop_out_of_scope(course_2019_2020)
year_2022_2023 = drop_out_of_scope(course_2022_2023)

In [13]:
def apply(year):
    df = split_term_year(year)
    df = split_term(df)
    df = assign_yrs(df)
    df = drop_unneeded_cols(df)
    df = group_by_year(df)
    df = drop_non_stream(df)
    df = reassign_seng(df)
    
    return df

In [14]:
df_2019_2020 = apply(year_2019_2020)
df_2022_2023 = apply(year_2022_2023)

df_2019_2020.drop(columns='Subj', inplace=True)
df_2022_2023.drop(columns='Subj', inplace=True)

/var/folders/64/crkcwxws21s3q44w2l4bghy40000gn/T/ipykernel_23603/4035378295.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Class Yr'] = df['Class Yr'].astype(int)
/var/folders/64/crkcwxws21s3q44w2l4bghy40000gn/T/ipykernel_23603/4035378295.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Course'] = df.apply(lambda row: row['Subj'] + ' ' + row['Num'], axis=1)
/var/folders/64/crkcwxws21s3q44w2l4bghy40000gn/T/ipykernel_23603/4035378295.py:9: SettingWithCopyWarning: 
A value is trying to be set 

In [17]:
df_2019 = df_2019_2020[df_2019_2020['Term Yr'] == 2019]
df_2020 = df_2019_2020[df_2019_2020['Term Yr'] == 2020]
df_2022 = df_2022_2023[df_2022_2023['Term Yr'] == 2022]
df_2023 = df_2022_2023[df_2022_2023['Term Yr'] == 2023]

In [18]:
df_2019_2020 = pd.concat([df_2019, df_2020])
df_2022_2023 = pd.concat([df_2022, df_2023])
df_all = pd.concat([df_2019_2020, df_2022_2023])
df_all.to_csv('../data/course_cleaned.csv', index=False)